In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.merge import concatenate
from keras.optimizers import SGD
from keras.models import load_model
from keras import regularizers

D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [2]:
df = pd.read_csv('./data.csv')

In [3]:
train_df = df[pd.to_datetime(df['quarter'])<="2019-01-01"]
test_df = df[pd.to_datetime(df['quarter'])=="2019-04-01"]

In [4]:
train_features=train_df.copy()
test_features=test_df.copy()
train_mret=train_df[['mret_net_weighted_qt_L1','mret_net_weighted_qt_L2','mret_net_weighted_qt_L3','mret_net_weighted_qt_L4']]
test_mret=test_df[['mret_net_weighted_qt_L1','mret_net_weighted_qt_L2','mret_net_weighted_qt_L3','mret_net_weighted_qt_L4']]
train_labels = train_features.pop('weight')
test_labels = test_features.pop('weight')
del train_features['mret_net_weighted_qt_L1']
del train_features['mret_net_weighted_qt_L2']
del train_features['mret_net_weighted_qt_L3']
del train_features['mret_net_weighted_qt_L4']
del test_features['mret_net_weighted_qt_L1']
del test_features['mret_net_weighted_qt_L2']
del test_features['mret_net_weighted_qt_L3']
del test_features['mret_net_weighted_qt_L4']
del train_features['quarter']
del test_features['quarter']

In [5]:
train_features=np.array(train_features)
test_features=np.array(test_features)
train_labels=np.array(train_labels)
test_labels=np.array(test_labels)
train_mret=np.array(train_mret)
test_mret=np.array(test_mret)

In [7]:
archi = [32, 16, 8]
dropout_u = 0.3



#Split Data for Test and Training
X_train = train_features
Xexog_train = train_mret
Y_train = train_labels

X_test = test_features
Xexog_test = test_mret
Y_test = test_labels

#Scale the predictors for training
Xscaler_train =  MinMaxScaler(feature_range=(-1,1))
X_scaled_train = Xscaler_train.fit_transform(X_train)

Xexog_scaler_train =  MinMaxScaler(feature_range=(-1,1))
Xexog_scaled_train = Xexog_scaler_train.fit_transform(Xexog_train)

# Scale the data for testing using the in-sample transformation from earlier
X_scaled_test = Xscaler_train.transform(X_test)
Xexog_scaled_test = Xexog_scaler_train.transform(Xexog_test)
# Keras requires 3D tuples for training.
X_scaled_train = np.expand_dims(X_scaled_train, axis=1)
Xexog_scaled_train = np.expand_dims(Xexog_scaled_train, axis=1)

Y_train = np.expand_dims(Y_train, axis=1)
Y_train = np.expand_dims(Y_train, axis=1)

Y_test = np.expand_dims(Y_test, axis=1)
Y_test = np.expand_dims(Y_test, axis=1)

# seed numpy and tf
tf.set_random_seed(123)
np.random.seed(123)


# Define Model Architecture

# Base model for macro variables
n = len(archi)
layers = dict()
for i in range(n+1):
    if i == 0:
        layers['ins_main'] = Input(shape=(1,X_scaled_train.shape[2]))
    elif i == 1:
        layers['dropout'+str(i)] = Dropout(dropout_u)(layers['ins_main'])
        layers['hidden'+str(i)] = Dense(archi[i-1],kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.001),bias_initializer='he_normal', 
                                        kernel_initializer='he_normal', activation='relu')(layers['dropout'+str(i)])
    elif i > 1 & i <= n:
        layers['dropout'+str(i)] = Dropout(dropout_u)(layers['hidden'+str(i-1)])
        layers['hidden'+str(i)] = Dense(archi[i-1],kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.001),bias_initializer='he_normal', 
                                        kernel_initializer='he_normal', activation='relu')(layers['dropout'+str(i)])

# Model for yield variables
layers['ins_exog'] = Input(shape=(1,Xexog_scaled_train.shape[2]))

# Merge macro / yield networks
layers['merge'] = concatenate([layers['hidden'+str(n)], layers['ins_exog']])
layers['dropout_final'] = Dropout(dropout_u)(layers['merge'])
layers['BN'] = BatchNormalization()(layers['dropout_final'])
layers['output'] = Dense(Y_train.shape[2],bias_initializer='he_normal',
                         kernel_initializer='he_normal')(layers['BN'])

model = Model(inputs=[layers['ins_main'], layers['ins_exog']], outputs=layers['output'])

# Compile model
sgd_fine = SGD(lr=0.01, momentum=0.9, decay=0.01, nesterov=True)
earlystopping = EarlyStopping(monitor='val_loss',min_delta=1e-6,
                              patience=20,verbose=0, mode='auto')

model.summary()
model.compile(loss='mean_squared_error', optimizer=sgd_fine)
history=model.fit([X_scaled_train, Xexog_scaled_train] , Y_train, epochs=300,
                  validation_split=0.15, batch_size=32, shuffle=True, verbose=1)



# Scale the data for testing using the in-sample transformation from earlier
X_scaled_test = Xscaler_train.transform(X_test)
Xexog_scaled_test = Xexog_scaler_train.transform(Xexog_test)
X_scaled_test = np.expand_dims(X_scaled_test, axis=1)
Xexog_scaled_test = np.expand_dims(Xexog_scaled_test, axis=1)

# Make out-of-sample prediction on the unseen observations
Ypred = model.predict([X_scaled_test,Xexog_scaled_test])
Ypred = np.squeeze(Ypred,axis=1)

score = model.evaluate([X_scaled_test,Xexog_scaled_test], Y_test, verbose=1)
print("Test score:", score)


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 45)        0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1, 45)        0           input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1, 32)        1472        dropout_1[0][0]                  
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 1, 32)        0           dense_1[0][0]                    
____________________________________________________________________________________________

17203/17203 [==============================] - 1s 42us/step - loss: 1.8445 - val_loss: 1.7282
Epoch 48/300
17203/17203 [==============================] - 1s 38us/step - loss: 1.8438 - val_loss: 1.7279
Epoch 49/300
17203/17203 [==============================] - 1s 39us/step - loss: 1.8434 - val_loss: 1.7276
Epoch 50/300
17203/17203 [==============================] - 1s 38us/step - loss: 1.8432 - val_loss: 1.7273
Epoch 51/300
17203/17203 [==============================] - 1s 37us/step - loss: 1.8429 - val_loss: 1.7267
Epoch 52/300
17203/17203 [==============================] - 1s 37us/step - loss: 1.8426 - val_loss: 1.7264
Epoch 53/300
17203/17203 [==============================] - 1s 39us/step - loss: 1.8423 - val_loss: 1.7262
Epoch 54/300
17203/17203 [==============================] - 1s 44us/step - loss: 1.8418 - val_loss: 1.7257
Epoch 55/300
17203/17203 [==============================] - 1s 41us/step - loss: 1.8417 - val_loss: 1.7257
Epoch 56/300
17203/17203 [========================

17203/17203 [==============================] - 1s 44us/step - loss: 1.8342 - val_loss: 1.7186
Epoch 124/300
17203/17203 [==============================] - 1s 43us/step - loss: 1.8341 - val_loss: 1.7185
Epoch 125/300
17203/17203 [==============================] - 1s 39us/step - loss: 1.8344 - val_loss: 1.7185
Epoch 126/300
17203/17203 [==============================] - 1s 37us/step - loss: 1.8341 - val_loss: 1.7184
Epoch 127/300
17203/17203 [==============================] - 1s 35us/step - loss: 1.8341 - val_loss: 1.7184
Epoch 128/300
17203/17203 [==============================] - 1s 38us/step - loss: 1.8341 - val_loss: 1.7184
Epoch 129/300
17203/17203 [==============================] - 1s 34us/step - loss: 1.8341 - val_loss: 1.7183
Epoch 130/300
17203/17203 [==============================] - 1s 37us/step - loss: 1.8342 - val_loss: 1.7183
Epoch 131/300
17203/17203 [==============================] - 1s 36us/step - loss: 1.8339 - val_loss: 1.7183
Epoch 132/300
17203/17203 [===============

17203/17203 [==============================] - 1s 40us/step - loss: 1.8331 - val_loss: 1.7173
Epoch 199/300
17203/17203 [==============================] - 1s 43us/step - loss: 1.8335 - val_loss: 1.7174
Epoch 200/300
17203/17203 [==============================] - 1s 44us/step - loss: 1.8333 - val_loss: 1.7174
Epoch 201/300
17203/17203 [==============================] - ETA: 0s - loss: 1.834 - 1s 39us/step - loss: 1.8333 - val_loss: 1.7173
Epoch 202/300
17203/17203 [==============================] - 1s 39us/step - loss: 1.8331 - val_loss: 1.7173
Epoch 203/300
17203/17203 [==============================] - 1s 39us/step - loss: 1.8332 - val_loss: 1.7172
Epoch 204/300
17203/17203 [==============================] - 1s 39us/step - loss: 1.8329 - val_loss: 1.7172
Epoch 205/300
17203/17203 [==============================] - 1s 39us/step - loss: 1.8332 - val_loss: 1.7173
Epoch 206/300
17203/17203 [==============================] - 1s 39us/step - loss: 1.8331 - val_loss: 1.7173
Epoch 207/300
1720

17203/17203 [==============================] - 1s 36us/step - loss: 1.8330 - val_loss: 1.7171
Epoch 274/300
17203/17203 [==============================] - 1s 35us/step - loss: 1.8329 - val_loss: 1.7171
Epoch 275/300
17203/17203 [==============================] - 1s 36us/step - loss: 1.8325 - val_loss: 1.7171
Epoch 276/300
17203/17203 [==============================] - 1s 36us/step - loss: 1.8324 - val_loss: 1.7171
Epoch 277/300
17203/17203 [==============================] - 1s 36us/step - loss: 1.8330 - val_loss: 1.7171
Epoch 278/300
17203/17203 [==============================] - 1s 36us/step - loss: 1.8330 - val_loss: 1.7171
Epoch 279/300
17203/17203 [==============================] - 1s 36us/step - loss: 1.8329 - val_loss: 1.7171
Epoch 280/300
17203/17203 [==============================] - 1s 36us/step - loss: 1.8332 - val_loss: 1.7171
Epoch 281/300
17203/17203 [==============================] - 1s 37us/step - loss: 1.8327 - val_loss: 1.7171
Epoch 282/300
17203/17203 [===============